In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import torch.nn.functional as F

In [13]:
from torch import nn

In [3]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [6]:
def get_data():
    mnist_url='http://deeplearning.net/data/mnist/mnist.pkl'
    path = datasets.download_data(mnist_url,ext='.gz')
    with gzip.open(path,'rb') as f:
        ((x_train, y_train), (x_valid,y_valid), _) = pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train,y_train,x_valid,y_valid))
def normalize(x,m,s):
    return (x-m)/s
mpl.rcParams['image.cmap'] = 'gray'

In [88]:
x_train,y_train,x_valid,y_valid = get_data()

In [89]:
n,features = x_train.shape

In [90]:
c = y_train.max()+1
dims = 50

In [8]:
y_train = y_train.float()
y_valid = y_valid.float()
m,s = x_train.mean(),x_train.std()
x_train = normalize(x_train,m,s)
x_valid = normalize(x_valid,m,s)

In [265]:
class Red(nn.Module):
    def __init__(self,n_in,dims,n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in,dims)
        self.l2 = nn.Linear(dims,n_out)
    def __call__(self,x):        
        return self.l2(F.relu(self.l1(x)))

In [278]:
class RedSeq(nn.Module):
    def __init__(self,layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
                
    def __call__(self,x):        
        for l in self.layers:
            x = l(x)
        return x

In [279]:
#model = Red(features,dims,10)
model = RedSeq([nn.Linear(features,dims),nn.ReLU(),nn.Linear(dims,10)])

In [280]:
pred=model(x_train)

In [281]:
def logsoftmax(x):
    #esto es por error numerico
    #  logexpsum
    #  m = x.max(-1)[0]    
    #  (m+(x-m[:,None]).exp().sum(-1).log())
    return x - x.logsumexp(-1,keepdim=True)
def nll(y_hat,y):
    return -y_hat[range(y.shape[0]),y].mean()

In [282]:
logsoftmax(x)

tensor([[-2.1056, -2.4495, -2.4392, -2.2497, -2.4000, -2.3233, -2.3163, -2.1620,
         -2.1882, -2.4685],
        [-2.2382, -2.3539, -2.4643, -2.2742, -2.3988, -2.2482, -2.3904, -2.1071,
         -2.2030, -2.4043],
        [-2.2220, -2.4391, -2.3489, -2.3275, -2.3357, -2.3011, -2.3410, -2.1813,
         -2.1786, -2.3844],
        [-2.2088, -2.4186, -2.3962, -2.2894, -2.4180, -2.2605, -2.3571, -2.1416,
         -2.2086, -2.3718],
        [-2.2101, -2.3163, -2.3080, -2.3713, -2.4250, -2.3472, -2.3855, -2.0497,
         -2.2597, -2.4130]], grad_fn=<SubBackward0>)

# PYTORCH TIENE F.nnl_loss y F.log_softmax que forman
# F.cross_entropy

In [298]:
def accuracy(y_hat,y):
    return (torch.argmax(y_hat,dim=1) == y).float().mean()
loss_f = F.cross_entropy
bs = 64

In [299]:
xb = x_train[0:bs]
preds = model(xb)
preds[0],preds.shape

(tensor([ -6.3996,   5.6154,  -1.2964,  24.1664, -31.0383,  32.2412, -17.6279,
          -5.9492, -10.5892,   6.4689], grad_fn=<SelectBackward>),
 torch.Size([64, 10]))

In [300]:
yb = y_train[0:bs]
loss_f(preds,yb)

tensor(0.0046, grad_fn=<NllLossBackward>)

In [301]:
accuracy(preds,yb)

tensor(1.)

In [302]:
class Optimizer():
    def __init__(self,params,lr = 0.5):
        self.params,self.lr = list(params),lr
        
    def step(self):
        #no calcules derivadas!
        with torch.no_grad():
            for p in self.params:
                p-=p.grad * lr
    def zero_grad(self):
        for p in self.params:
            p.grad.data.zero_()

In [303]:
opt = Optimizer(model.parameters())

In [304]:
lr = .5
epochs = 10

In [305]:
for epoch in range(epochs):
    for i in range(0,x_train.shape[0],bs):
        xb = x_train[i:i+bs]
        preds = model(xb)
        
        yb = y_train[i:i+bs]                
        loss = loss_f(preds,yb)
        
        loss.backward()
        
        opt.step()
        opt.zero_grad()        

In [306]:
accuracy(preds,yb)

tensor(1.)

In [307]:
torch.argmax(preds,dim=1)

tensor([9, 3, 7, 0, 9, 0, 8, 5, 5, 2, 4, 5, 0, 8, 4, 8])

In [261]:
yb

tensor([9, 3, 7, 0, 9, 0, 8, 5, 5, 2, 4, 5, 0, 8, 4, 8])